<a href="https://colab.research.google.com/github/ak-hannou/compsci-4ml3/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import torch  # Core PyTorch library for tensor operations
from torchvision import datasets
from torchvision.transforms import v2  # Datasets and transformations for computer vision
import torch.nn as nn  # Neural network components
import torch.optim as optim  # Optimization algorithms
import numpy as np  # Numerical operations
import matplotlib.pyplot as plt  # Plotting
from torch.utils.data import DataLoader, Dataset, random_split, Subset, TensorDataset  # Data handling utilities
import pandas as pd

In [54]:
batch_size = 32 #
learning_rate = 0.001      # Learning rate for optimizer
num_epochs = 100

In [55]:
cifar100_mean = (0.5071, 0.4867, 0.4408)
cifar100_std = (0.2675, 0.2565, 0.2761)

train_transform = v2.Compose([
    v2.RandomHorizontalFlip(),
    # v2.RandomVerticalFlip(),
    # v2.RandomZoomOut(),
    v2.RandomRotation(15),
    # v2.RandomGrayscale(),
    # v2.RandomResizedCrop(32),
      v2.ToTensor(),
      v2.Normalize(cifar100_mean, cifar100_std),
      ###### Add your transformations here ########
])
test_transform = v2.Compose([
      v2.ToTensor(),
      v2.Normalize(cifar100_mean, cifar100_std),
      ###### Add your transformations here ########
])

train_dataset = datasets.CIFAR100(
    root='./data',       # Change this path if needed
    train=True,          # Set to True to download the training set
    download=True,       # Set to True to download if not already downloaded
    transform=train_transform  # Apply transformations
)
test_dataset = datasets.CIFAR100(
    root='./data',       # Change this path if needed
    train=False,          # Set to True to download the training set
    download=True,       # Set to True to download if not already downloaded
    transform=test_transform  # Apply transformations
)
##### Hyper-parameters


# Create a DataLoader for batch processing
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,      # Batch size (you can modify this as needed)
    shuffle=True        # Shuffle data for training
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,      # Batch size (you can modify this as needed)
    shuffle=False        # Shuffle data for training
)


Files already downloaded and verified
Files already downloaded and verified


In [56]:
class CIFARClassifier(nn.Module):
    def __init__(self):
        super(CIFARClassifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1, stride=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.4)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, stride=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.3)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1, stride=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout3 = nn.Dropout(0.25)

        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1, stride=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout4 = nn.Dropout(0.25)

        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1, stride=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.relu5 = nn.ReLU()

        self.fc1 = nn.Linear(512*2*2, 256)
        self.relu6 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 100)

    def forward(self, x):

      x = self.conv1(x)
      x = self.bn1(x)
      x = self.relu1(x)
      x = self.pool1(x)
      x = self.dropout1(x)

      x = self.conv2(x)
      x = self.bn2(x)
      x = self.relu2(x)
      x = self.pool2(x)
      x = self.dropout2(x)

      x = self.conv3(x)
      x = self.bn3(x)
      x = self.relu3(x)
      x = self.pool3(x)
      x = self.dropout3(x)

      x = self.conv4(x)
      x = self.bn4(x)
      x = self.relu4(x)
      x = self.pool4(x)
      x = self.dropout4(x)

      x = self.conv5(x)
      x = self.bn5(x)
      x = self.relu5(x)
      #x = self.pool5(x)

      x = x.view(x.size(0), -1)

      x = self.fc1(x)
      x = self.relu6(x)
      x = self.dropout(x)
      x = self.fc2(x)
      return x


In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CIFARClassifier().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # CrossEntropy includes softmax
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-6)

In [58]:
def evaluate(model, data_loader, criterion):
  model.eval()
  correct = 0
  total = 0
  loss_batches = []

  # Switch to evaluation mode and turn off gradient calculation
  # since parameters are not updated during testing.
  with torch.no_grad():
      for images_batch, labels_batch in data_loader:
          images_batch, labels_batch = images_batch.to(device), labels_batch.to(device)
          outputs = model(images_batch) # Forward pass
          # The predicted label is the output with the highest activation.
          _, predicted = torch.max(outputs.data, 1)
          total += labels_batch.size(0)
          correct += (predicted == labels_batch).sum().item()

          # Use provided criterion to calculate the loss for the mini batch
          # Append the mini-batch loss to loss_batches array
          batch_loss = criterion(outputs, labels_batch)
          loss_batches.append(batch_loss.item())

      accuracy = 100 * correct / total
      avg_loss = np.mean(loss_batches)

      model.train()


      return accuracy, avg_loss

In [59]:
# Training the model
train_losses, test_losses = [], []
train_accuracies, test_accuracies = [], []

for epoch in range(num_epochs):
    for i, (images_batch, labels_batch) in enumerate(train_loader):
        optimizer.zero_grad() # Clear the gradients
        images_batch, labels_batch = images_batch.to(device), labels_batch.to(device)
        outputs = model(images_batch) # Forward pass
        loss = criterion(outputs, labels_batch) # Calculate loss
        loss.backward() # Backward pass
        optimizer.step() # Update weights

    # Evaluate on train and test sets after each epoch

    train_accuracy, train_loss = evaluate(model, train_loader, criterion)
    test_accuracy, test_loss = evaluate(model, test_loader, criterion)


    print(f'Epoch {epoch+1:02d}/{num_epochs:02d} - Train Loss: {train_loss:.6f}, Train Acc: {train_accuracy:.2f}%')
    print(f'            - Test Loss: {test_loss:.6f}, Test Acc: {test_accuracy:.2f}%')
    print("-" * 60)

    train_losses.append(train_loss)
    test_losses.append(test_loss)
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)

Epoch 01/100 - Train Loss: 4.096209, Train Acc: 5.60%
            - Test Loss: 4.101091, Test Acc: 6.14%
------------------------------------------------------------
Epoch 02/100 - Train Loss: 3.848285, Train Acc: 9.09%
            - Test Loss: 3.856274, Test Acc: 9.61%
------------------------------------------------------------
Epoch 03/100 - Train Loss: 3.733185, Train Acc: 10.40%
            - Test Loss: 3.735750, Test Acc: 10.70%
------------------------------------------------------------
Epoch 04/100 - Train Loss: 3.606571, Train Acc: 12.98%
            - Test Loss: 3.608780, Test Acc: 13.53%
------------------------------------------------------------
Epoch 05/100 - Train Loss: 3.506551, Train Acc: 14.98%
            - Test Loss: 3.498977, Test Acc: 15.56%
------------------------------------------------------------
Epoch 06/100 - Train Loss: 3.355623, Train Acc: 18.13%
            - Test Loss: 3.354712, Test Acc: 17.88%
---------------------------------------------------------

In [62]:
# Load test.csv
test_data = pd.read_csv("test.csv")

# Extract IDs and image data
test_ids = test_data["ID"]
image_data = test_data.drop(columns=["ID"]).values

# Reshape to (num_samples, 3, 32, 32) without normalization
num_samples = image_data.shape[0]
images = image_data.reshape(num_samples, 3, 32, 32).astype('float32')  # Assuming it's normalized already

# Convert to PyTorch tensor
images_tensor = torch.tensor(images)

In [63]:
test_dataset = TensorDataset(images_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [64]:
model.eval()

predictions = []
with torch.no_grad():
    for images_batch in test_loader:
        images_batch = images_batch[0].to(device)  # Move to GPU/CPU
        outputs = model(images_batch)  # Forward pass
        predicted_labels = torch.argmax(outputs, dim=1)  # Predicted class
        predictions.extend(predicted_labels.cpu().numpy())  # Collect predictions

In [65]:
# Combine predictions with IDs
submission = pd.DataFrame({
    "ID": test_ids,
    "Label": predictions
})

# Save predictions to CSV
submission.to_csv("submission.csv", index=False)
print("Predictions saved to submission.csv")

Predictions saved to submission.csv
